<a href="https://colab.research.google.com/github/HaJunYoo/Audio_Speech_Processing_Study/blob/main/%08spectrogram_%EC%A0%80%EC%9E%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
import os 
path = '/content/drive/MyDrive/졸플/Audio_study/'

In [3]:
import librosa
import numpy as np

# Define parameters for the sliding window approach
window_size = 0.02 # seconds
hop_size = 0.01 # seconds
n_fft = 2048

# Load an example audio file
audio, sr = librosa.load(os.path.join(path, '1.강제추행(성범죄)_17951_label.wav'), sr=16000)


In [4]:
# Compute the length of the audio file in seconds and frames
audio_len = len(audio) / sr
audio_frames = int(audio_len / hop_size)

In [5]:
# Initialize an empty array to store the spectrogram frames
spectrogram_frames = np.zeros((audio_frames, n_fft // 2 + 1))

In [6]:
# Generate the spectrogram frames using a sliding window approach
for i in range(audio_frames):
    start = int(i * hop_size * sr)
    end = int(start + window_size * sr)
    window = audio[start:end]
    spectrogram = np.abs(librosa.stft(window, n_fft=n_fft))
    spectrogram_frames[i] = spectrogram.mean(axis=1)


/usr/local/lib/python3.8/dist-packages/librosa/core/spectrum.py:222: UserWarning: n_fft=2048 is too small for input signal of length=320
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/librosa/core/spectrum.py:222: UserWarning: n_fft=2048 is too small for input signal of length=319
  warnings.warn(


In [7]:
# Reshape the spectrogram frames to have a single channel
spectrogram_frames = spectrogram_frames.reshape(spectrogram_frames.shape[0], spectrogram_frames.shape[1], 1)

In [9]:
spectrogram_frames.shape

(238, 1025, 1)

## 가변적 크기의 스펙트로그램

In [18]:
import os
import librosa
import librosa.display
import matplotlib.pyplot as plt

wav_dir = os.path.join(path,'data')

spect_dir = os.path.join(path, 'spectrogram')
if not os.path.exists(spect_dir):
    os.makedirs(spect_dir)

# 디렉토리 내의 모든 wav 파일들을 불러옵니다.
for wav_file in os.listdir(wav_dir):
    if wav_file.endswith('.wav'):
        # wav 파일을 로드합니다.
        audio_path = os.path.join(wav_dir, wav_file)
        y, sr = librosa.load(audio_path)

        # 스펙트로그램을 생성합니다.
        spect = librosa.feature.melspectrogram(y=y, sr=sr)

        # 생성된 스펙트로그램을 시각화하고 저장합니다.
        plt.figure(figsize=(10, 4))
        librosa.display.specshow(librosa.power_to_db(spect, ref=np.max),
                                 y_axis='mel', fmax=8000,
                                 x_axis='time')
        
        plt.colorbar(format='%+2.0f dB')
        plt.title(wav_file)
        plt.tight_layout()

        # 저장될 파일 경로를 생성합니다.
        spect_file = os.path.join(spect_dir, os.path.splitext(wav_file)[0] + '.png')

        # 스펙트로그램 이미지를 저장합니다.
        plt.savefig(spect_file, dpi=100)
        plt.close()

<ipython-input-18-3cccd62c1247>:23: UserWarning: Glyph 4352 (\N{HANGUL CHOSEONG KIYEOK}) missing from current font.
  plt.tight_layout()
<ipython-input-18-3cccd62c1247>:23: UserWarning: Glyph 4449 (\N{HANGUL JUNGSEONG A}) missing from current font.
  plt.tight_layout()
<ipython-input-18-3cccd62c1247>:23: UserWarning: Glyph 4540 (\N{HANGUL JONGSEONG IEUNG}) missing from current font.
  plt.tight_layout()
<ipython-input-18-3cccd62c1247>:23: UserWarning: Glyph 4364 (\N{HANGUL CHOSEONG CIEUC}) missing from current font.
  plt.tight_layout()
<ipython-input-18-3cccd62c1247>:23: UserWarning: Glyph 4454 (\N{HANGUL JUNGSEONG E}) missing from current font.
  plt.tight_layout()
<ipython-input-18-3cccd62c1247>:23: UserWarning: Glyph 4366 (\N{HANGUL CHOSEONG CHIEUCH}) missing from current font.
  plt.tight_layout()
<ipython-input-18-3cccd62c1247>:23: UserWarning: Glyph 4462 (\N{HANGUL JUNGSEONG U}) missing from current font.
  plt.tight_layout()
<ipython-input-18-3cccd62c1247>:23: UserWarning: Glyp

## 고정된 크기의 스펙트로 그램

In [21]:
# 고정된 크기로 spectrogram을 생성
import os
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt

# Step 1: Preprocess the audio files and save the spectrograms as images

# Define the fixed shape of the spectrograms
# A common choice is to set the height to 128 pixels and the width to 128-256 pixels
img_height, img_width = 128, 256

path = '/content/drive/MyDrive/졸플/Audio_study/'

# Define the input directory for the audio files
audio_dir = os.path.join(path,'data')

# Define the output directory for the spectrogram images
spec_dir = os.path.join(path,'spectrogram_fixed')
if not os.path.exists(spec_dir):
    os.makedirs(spec_dir)

# Loop over all the audio files in the input directory
for filename in os.listdir(audio_dir):
    if filename.endswith('.wav'):
        # Load the audio file
        filepath = os.path.join(audio_dir, filename)
        y, sr = librosa.load(filepath, sr=22050)

        # Generate the spectrogram
        S = librosa.feature.melspectrogram(y=y, sr=sr)
        S_db = librosa.power_to_db(S, ref=np.max)

        # Resize the spectrogram to the fixed shape
        S_resized = librosa.util.fix_length(S_db, img_width)

        # Save the spectrogram as an image file
        spec_filename = filename[:-4] + '.png'
        spec_filepath = os.path.join(spec_dir, spec_filename)
        plt.imsave(spec_filepath, S_resized)
